In [37]:
from sedona.spark import *
from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, StringType
from pyspark.sql.functions import col, regexp_replace, when, isnull, isnan, sum, count, upper, round
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import time

spark = SparkSession.builder \
    .appName("Query 3") \
    .getOrCreate()

sedona = SedonaContext.create(spark)

blocks_path = "s3://initial-notebook-data-bucket-dblab-905418150721/2010_Census_Blocks.geojson"
income_path = "s3://initial-notebook-data-bucket-dblab-905418150721/LA_income_2015.csv"

blocks_df = sedona.read.format("geojson") \
            .option("multiLine", "true").load(blocks_path) \
            .selectExpr("explode(features) as features") \
            .select("features.*")

blocks_df = blocks_df.select( \
                [col(f"properties.{col_name}").alias(col_name) for col_name in \
                blocks_df.schema["properties"].dataType.fieldNames()] + ["geometry"]) \
            .drop("properties") \
            .drop("type")
# κραταμε μονο τις στηλες που μας ενδιαφερουν και τις γραμμες εκεινες
# που εχουν να κανουν με τον δημο του Los Angeles
blocks_df = blocks_df.select("CITY", "COMM", "HOUSING10", "POP_2010")
blocks_df = blocks_df.filter(col("CITY").contains("Los Angeles"))
# υπαρχουν περιοχες που περιεχουν παραπανω απο ενα ZIP code οποτε κανουμε
# goupby με βαση τη καθε περιοχη και προσθετουμε τους επιμερους πληθυσμους
# και τα επιμερους νοικοκυρια
df1 = blocks_df.groupBy("COMM") \
                .agg(sum("POP_2010").alias("total_population"),
                     sum("HOUSING10").alias("number_of_houses"))
# διαβαζουμε το csv με το μεσο εισοδημα ανα νοικοκυριο και κραταμε
# μονο τις στηλες που μας ενδιαφερουν και μονο τις περιπτωσεις
# εκεινες οπου η περιοχη ανηκει στον δημο του Los Angeles
df2 = spark.read.csv(income_path, header=True, inferSchema=True)
df2 = df2.select("Community", "Estimated Median Income")
df2 = df2.filter(col("Community").contains("Los Angeles"))
# αφαιρουμε το συμβολο του δολαριου απο την στηλη με το εισοδημα
# και μετατρεπουμε τον τυπο σε int
df2 = df2.withColumn("int_income",
                     regexp_replace(col("Estimated Median Income"),
                                    r"[$,]", "").cast(IntegerType()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
# κανουμε join υπο την συνθηκη οτι το περιεχομενο του COMM του
# ενος συνολου δεδομενων να περιεχεται στην στηλη Community του
# αλλου συνολου, ετσι ωστε να εχουμε κοινες περιοχες
start_time = time.time()
df = df1.hint("broadcast").join(df2, df2.Community.contains(df1.COMM), "inner")
df.show()
df.explain(mode="formatted")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Broadcast Time: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+----------------+----------------+--------------------+-----------------------+----------+
|           COMM|total_population|number_of_houses|           Community|Estimated Median Income|int_income|
+---------------+----------------+----------------+--------------------+-----------------------+----------+
|          Watts|           39657|            9802|Los Angeles (Sout...|                $30,413|     30413|
|Wilshire Center|           47295|           19835|Los Angeles (Hanc...|                $40,612|     40612|
|   Hancock Park|           15557|            6653|Los Angeles (Hanc...|                $40,612|     40612|
|Wilshire Center|           47295|           19835|Los Angeles (Hanc...|                $31,142|     31142|
|      Koreatown|           48275|           19863|Los Angeles (Hanc...|                $31,142|     31142|
|   Hancock Park|           15557|            6653|Los Angeles (Hanc...|                $31,142|     31142|
|      Koreatown|           

In [39]:
# κανουμε join υπο την συνθηκη οτι το περιεχομενο του COMM του
# ενος συνολου δεδομενων να περιεχεται στην στηλη Community του
# αλλου συνολου, ετσι ωστε να εχουμε κοινες περιοχες
start_time = time.time()
df = df1.hint("merge").join(df2, df2.Community.contains(df1.COMM), "inner")
df.show()
df.explain(mode="formatted")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Merge Time: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+----------------+----------------+--------------------+-----------------------+----------+
|           COMM|total_population|number_of_houses|           Community|Estimated Median Income|int_income|
+---------------+----------------+----------------+--------------------+-----------------------+----------+
| Toluca Terrace|            1301|             541|Los Angeles (Nort...|                $48,499|     48499|
|Lincoln Heights|           31144|            9316|Los Angeles (Linc...|                $34,994|     34994|
|       Van Nuys|           86019|           29170|Los Angeles (Vall...|                $50,418|     50418|
|       Van Nuys|           86019|           29170|Los Angeles (Sher...|                $80,890|     80890|
|       Van Nuys|           86019|           29170|Los Angeles (Lake...|                $49,835|     49835|
|       Van Nuys|           86019|           29170|Los Angeles (Van ...|                $45,667|     45667|
|       Van Nuys|           

In [40]:
# κανουμε join υπο την συνθηκη οτι το περιεχομενο του COMM του
# ενος συνολου δεδομενων να περιεχεται στην στηλη Community του
# αλλου συνολου, ετσι ωστε να εχουμε κοινες περιοχες
start_time = time.time()
df = df1.hint("shuffle_hash").join(df2, df2.Community.contains(df1.COMM), "inner")
df.show()
df.explain(mode="formatted")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Shuffle_hash Time: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+----------------+----------------+--------------------+-----------------------+----------+
|           COMM|total_population|number_of_houses|           Community|Estimated Median Income|int_income|
+---------------+----------------+----------------+--------------------+-----------------------+----------+
| Toluca Terrace|            1301|             541|Los Angeles (Nort...|                $48,499|     48499|
|Lincoln Heights|           31144|            9316|Los Angeles (Linc...|                $34,994|     34994|
|       Van Nuys|           86019|           29170|Los Angeles (Vall...|                $50,418|     50418|
|       Van Nuys|           86019|           29170|Los Angeles (Sher...|                $80,890|     80890|
|       Van Nuys|           86019|           29170|Los Angeles (Lake...|                $49,835|     49835|
|       Van Nuys|           86019|           29170|Los Angeles (Van ...|                $45,667|     45667|
|       Van Nuys|           

In [41]:
# κανουμε join υπο την συνθηκη οτι το περιεχομενο του COMM του
# ενος συνολου δεδομενων να περιεχεται στην στηλη Community του
# αλλου συνολου, ετσι ωστε να εχουμε κοινες περιοχες
start_time = time.time()
df = df1.hint("shuffle_replicate_nl").join(df2, df2.Community.contains(df1.COMM), "inner")
df.show()
df.explain(mode="formatted")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Shuffle_replicate_nl Time: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+----------------+----------------+--------------------+-----------------------+----------+
|           COMM|total_population|number_of_houses|           Community|Estimated Median Income|int_income|
+---------------+----------------+----------------+--------------------+-----------------------+----------+
| Toluca Terrace|            1301|             541|Los Angeles (Nort...|                $48,499|     48499|
|Lincoln Heights|           31144|            9316|Los Angeles (Linc...|                $34,994|     34994|
|       Van Nuys|           86019|           29170|Los Angeles (Vall...|                $50,418|     50418|
|       Van Nuys|           86019|           29170|Los Angeles (Sher...|                $80,890|     80890|
|       Van Nuys|           86019|           29170|Los Angeles (Lake...|                $49,835|     49835|
|       Van Nuys|           86019|           29170|Los Angeles (Van ...|                $45,667|     45667|
|       Van Nuys|           

In [42]:
df = df.select("COMM", "total_population", "number_of_houses", "int_income")
# καθως ειχαμε περισσοτερα απο ενα ZIP code για μια περιοχη, ειχαμε
# περισσοτερα απο ενα μεσα εισοδηματα ανα νοικοκυριο, επομενως σκοπος
# ειναι σε ενα dataframe να εχουμε σε μια γραμμη την περιοχη, το συνολικο
# πληθος νοικοκυριων, το συνολικο πληθυσμο και το συνολικο μεσο εισοδημα
# ανα νοικοκυριο το οποιο θα προκυψει απο τον μεσο ορο ολων των επομερους
# μεσων εισοδηματων ανα νοικοκυριων
temp_df1 = df.groupBy("COMM").agg(
    sum("int_income").alias("sum_income"),
    count("*").alias("number of ZIPs"))
temp_df2 = df.select("COMM", "total_population", "number_of_houses").distinct()
df = temp_df1.join(temp_df2, on="COMM", how="inner")
df = df.withColumn("total median income", col("sum_income") / col("number of ZIPs"))
# υπολογισμος κατα κεφαλην εισοδηματος
df = df.withColumn("income per capita",
                   round((col("total median income") * col("number_of_houses")) / col("total_population"), 3))
# σε νεα dataframes κραταμε την περιοχη και τον συνολικο πληθυσμο απο την μια
# και την περιοχη με το κατα κεφαλην εισοδημα απο την αλλη
population_df = df.select("COMM", "total_population")
capita_df = df.select("COMM", "income per capita") \
                .sort("income per capita", ascending=False)

file_1 = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
file_2 = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"
# διαβαζουμε τα συνολα δεδομενων με τα εγκληματα και τα συνεννωνουμε
# σε ενα dataframe απο το οποιο βγαζουμε τις ακυρες περιπτωσεις με θυματα
# αρνητικη ηλικιας και κραταμε τις στηλες που μας ενδιαφερουν
df1 = spark.read.csv(file_1, header=True, inferSchema=True)
df2 = spark.read.csv(file_2, header=True, inferSchema=True)
df = df1.union(df2)
df = df.filter((col("Vict Age") > 0)) \
       .select("LOCATION", "Cross Street") \
       .withColumnRenamed("Cross Street", "cross_street")
# κανουμε ολους τους χαρακτηρες των περιοχων στο προηγουμενο dataframe
# κεφαλαιους για να κανουμε χρηση του contain και να βρουμε εκεινα τα
# εγκληματα που εγιναν στην αντιστοιχη περιοχη
population_df = population_df.withColumn("Locations",
                                         upper(population_df["COMM"])) \
                                .select("Locations", "total_population")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
# κανουμε join με παρομοιο τροπο οπως και πριν, για να εχουμε περιοχες
# με εγκληματα, η καθε περιοχη θα εμφανιζεται οσες φορες εχει διαπραχθει
# σε αυτη καποιο εκγλημα
start_time = time.time()
crime_df = population_df.hint("broadcast").join(df, df.LOCATION.contains(population_df.Locations) | df.cross_street.contains(population_df.Locations), "inner")
crime_df.show()
crime_df.explain(mode="formatted")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Broadcast Time: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----------------+--------------------+--------------------+
|Locations|total_population|            LOCATION|        cross_street|
+---------+----------------+--------------------+--------------------+
|HOLLYWOOD|           62412|CAHUENGA         ...|HOLLYWOOD        ...|
|SAN PEDRO|           73669|8TH              ...|SAN PEDRO        ...|
|SAN PEDRO|           73669|600 S  SAN PEDRO ...|                NULL|
|SAN PEDRO|           73669|SAN PEDRO        ...|7TH              ...|
|SAN PEDRO|           73669|500 S  SAN PEDRO ...|                NULL|
|SAN PEDRO|           73669|700 S  SAN PEDRO ...|                NULL|
|SAN PEDRO|           73669|    500 S  SAN PEDRO|                NULL|
|  CENTRAL|           35422|                 7TH|CENTRAL          ...|
|SAN PEDRO|           73669|500 S  SAN PEDRO ...|                NULL|
|SAN PEDRO|           73669|600 S  SAN PEDRO ...|                NULL|
|SAN PEDRO|           73669|600 S  SAN PEDRO ...|                NULL|
|SAN P

In [44]:
# κανουμε join με παρομοιο τροπο οπως και πριν, για να εχουμε περιοχες
# με εγκληματα, η καθε περιοχη θα εμφανιζεται οσες φορες εχει διαπραχθει
# σε αυτη καποιο εκγλημα
start_time = time.time()
crime_df = population_df.hint("merge").join(df, df.LOCATION.contains(population_df.Locations) | df.cross_street.contains(population_df.Locations), "inner")
crime_df.show()
crime_df.explain(mode="formatted")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Merge Time: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----------------+--------------------+--------------------+
|Locations|total_population|            LOCATION|        cross_street|
+---------+----------------+--------------------+--------------------+
|  CENTRAL|           35422|                 7TH|CENTRAL          ...|
|  CENTRAL|           35422|                 7TH|CENTRAL          ...|
|  CENTRAL|           35422|                 7TH|             CENTRAL|
|  CENTRAL|           35422|             CENTRAL|6TH              ...|
|  CENTRAL|           35422|2ND              ...|CENTRAL          ...|
|  CENTRAL|           35422|                 2ND|             CENTRAL|
|  CENTRAL|           35422|                 2ND|             CENTRAL|
|  CENTRAL|           35422|                 2ND|             CENTRAL|
|  CENTRAL|           35422|             CENTRAL|              N  6TH|
|  CENTRAL|           35422|                 7TH|             CENTRAL|
|  CENTRAL|           35422|             CENTRAL|7TH              ...|
|  CEN

In [45]:
# κανουμε join με παρομοιο τροπο οπως και πριν, για να εχουμε περιοχες
# με εγκληματα, η καθε περιοχη θα εμφανιζεται οσες φορες εχει διαπραχθει
# σε αυτη καποιο εκγλημα
start_time = time.time()
crime_df = population_df.hint("shuffle_hash").join(df, df.LOCATION.contains(population_df.Locations) | df.cross_street.contains(population_df.Locations), "inner")
crime_df.show()
crime_df.explain(mode="formatted")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Shuffle_hash Time: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----------------+--------------------+--------------------+
|Locations|total_population|            LOCATION|        cross_street|
+---------+----------------+--------------------+--------------------+
|  CENTRAL|           35422|                 7TH|CENTRAL          ...|
|  CENTRAL|           35422|                 7TH|CENTRAL          ...|
|  CENTRAL|           35422|                 7TH|             CENTRAL|
|  CENTRAL|           35422|             CENTRAL|6TH              ...|
|  CENTRAL|           35422|2ND              ...|CENTRAL          ...|
|  CENTRAL|           35422|                 2ND|             CENTRAL|
|  CENTRAL|           35422|                 2ND|             CENTRAL|
|  CENTRAL|           35422|                 2ND|             CENTRAL|
|  CENTRAL|           35422|             CENTRAL|              N  6TH|
|  CENTRAL|           35422|                 7TH|             CENTRAL|
|  CENTRAL|           35422|             CENTRAL|7TH              ...|
|  CEN

In [46]:
# κανουμε join με παρομοιο τροπο οπως και πριν, για να εχουμε περιοχες
# με εγκληματα, η καθε περιοχη θα εμφανιζεται οσες φορες εχει διαπραχθει
# σε αυτη καποιο εκγλημα
start_time = time.time()
crime_df = population_df.hint("shuffle_replicate_nl").join(df, df.LOCATION.contains(population_df.Locations) | df.cross_street.contains(population_df.Locations), "inner")
crime_df.show()
crime_df.explain(mode="formatted")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Shuffle_replicate_nl Time: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----------------+--------------------+--------------------+
|Locations|total_population|            LOCATION|        cross_street|
+---------+----------------+--------------------+--------------------+
|  CENTRAL|           35422|                 7TH|CENTRAL          ...|
|  CENTRAL|           35422|                 7TH|CENTRAL          ...|
|  CENTRAL|           35422|                 7TH|             CENTRAL|
|  CENTRAL|           35422|             CENTRAL|6TH              ...|
|  CENTRAL|           35422|2ND              ...|CENTRAL          ...|
|  CENTRAL|           35422|                 2ND|             CENTRAL|
|  CENTRAL|           35422|                 2ND|             CENTRAL|
|  CENTRAL|           35422|                 2ND|             CENTRAL|
|  CENTRAL|           35422|             CENTRAL|              N  6TH|
|  CENTRAL|           35422|                 7TH|             CENTRAL|
|  CENTRAL|           35422|             CENTRAL|7TH              ...|
|  CEN

In [47]:
# μετραμε τον αριθμο φορων εμφανισης μιας περιοχης και στοχευουμε
# σε ενα dataframe με περιοχη - πληθυσμο - αιρθμο εγκηλματων
temp_df1 = crime_df.groupBy("Locations").agg(count("*").alias("count"))
temp_df2 = crime_df.select("Locations", "total_population").distinct()
crime_df = temp_df1.join(temp_df2, on="Locations", how="inner")
# υπολογιζουμε την αναλογια συνολικου αριθμου εγκληματων ανα ατομο
crime_df = crime_df.withColumn("crime per capita",
                               round(col("count") / col("total_population"), 4)) \
                    .select("Locations", "crime per capita") \
                    .sort("crime per capita", ascending=False)
# μετατρεπουμε παλι σε κεφαλαια για την χρηση του contain
capita_df = capita_df.withColumn("LOCATIONS", upper(capita_df["COMM"])) \
                        .select("LOCATIONS", "income per capita")
result_df = capita_df.join(crime_df, on="LOCATIONS", how="inner")
result_df.sort("income per capita", ascending=False)

result_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----------------+----------------+
|   LOCATIONS|income per capita|crime per capita|
+------------+-----------------+----------------+
|SHADOW HILLS|        26640.871|          0.0034|
|   HOLLYWOOD|        32123.914|          0.3474|
|  CHATSWORTH|        30548.904|          0.1083|
|      VENICE|        46048.171|          0.3877|
|   HYDE PARK|        14137.654|          0.0322|
| WESTCHESTER|        33241.884|          0.0142|
| HARBOR CITY|        19581.889|          3.0E-4|
|      SYLMAR|        16356.332|          0.0323|
|PORTER RANCH|         35655.03|          0.0083|
| TOLUCA LAKE|        37679.301|          0.0062|
|    WESTLAKE|        10972.646|          0.0665|
|     CENTRAL|         4589.135|          0.3817|
|     MELROSE|        33511.914|          0.0937|
|   SAN PEDRO|        27737.601|          0.2786|
|      ENCINO|        44129.356|          0.0277|
|   ECHO PARK|        20745.656|          0.0947|
|   MAR VISTA|        31410.444|          0.0033|


In [48]:
# για αποθηκευση του κατα κεφαλην εισοδηματος
result_df.write \
    .option("header", "true") \
    .mode("overwrite") \
    .csv("s3://groups-bucket-dblab-905418150721/group25/query4/Income per capita/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…